In [1]:
cuda_code_file = "./src/gpu.cu"
header_code_file = "./src/gpu.hpp"

In [2]:
with open(cuda_code_file) as f:
    cuda_code = "".join([f for f in f.readlines() if not f.startswith("#include")])
    print(cuda_code)


void printCudaVersion()
{
    std::cout << "CUDA Compiled version: " << __CUDACC_VER_MAJOR__ << "." << __CUDACC_VER_MINOR__ << std::endl;

    int runtime_ver;
    cudaRuntimeGetVersion(&runtime_ver);
    std::cout << "CUDA Runtime version: " << runtime_ver << std::endl;

    int driver_ver;
    cudaDriverGetVersion(&driver_ver);
    std::cout << "CUDA Driver version: " << driver_ver << std::endl;
}

__global__
void saxpy(int n, float a, float *x, float *y) {
    int i = blockIdx.x*blockDim.x + threadIdx.x;
    if (i < n) {
        y[i] = a*x[i] + y[i];
    }
}

torch::Tensor saxpy_wrapper(const torch::Tensor& x, torch::Tensor y, float a) {
    auto n = static_cast<int32_t>(torch::numel(x));
    saxpy<<<n, 1>>>(n, a, x.data_ptr<float>(), y.data_ptr<float>());
    std::cout <<  "Calculated saxpy\n";
    cudaDeviceSynchronize();
    return y;
}


In [3]:
with open(header_code_file) as f:
    header_code = "".join([f for f in f.readlines() if not f.startswith("#include")])
    print(header_code)


void printCudaVersion();

torch::Tensor saxpy_wrapper(const torch::Tensor& x, torch::Tensor y, float a);



In [4]:
!rm ./build/*

In [5]:
import torch
from torch.utils.cpp_extension import load_inline

saxpy_extension = load_inline(
    name='saxpy_extension',
    cpp_sources=header_code,
    cuda_sources=cuda_code,
    functions=['saxpy_wrapper', "printCudaVersion"],
    with_cuda=True,
    verbose=True,
    extra_cuda_cflags=["-O2"],
    build_directory='./build',
    # extra_cuda_cflags=['--expt-relaxed-constexpr']
)

a = torch.tensor([[1., 2., 3.], [4., 5., 6.]], device='cuda')
saxpy_extension.printCudaVersion()

Detected CUDA files, patching ldflags
Emitting ninja build file ./build/build.ninja...
Building extension module saxpy_extension...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] c++ -MMD -MF main.o.d -DTORCH_EXTENSION_NAME=saxpy_extension -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/ksharma/anaconda3/envs/cuda-learn/lib/python3.12/site-packages/torch/include -isystem /home/ksharma/anaconda3/envs/cuda-learn/lib/python3.12/site-packages/torch/include/torch/csrc/api/include -isystem /home/ksharma/anaconda3/envs/cuda-learn/lib/python3.12/site-packages/torch/include/TH -isystem /home/ksharma/anaconda3/envs/cuda-learn/lib/python3.12/site-packages/torch/include/THC -isystem /home/ksharma/anaconda3/envs/cuda-learn/include -isystem /home/ksharma/anaconda3/envs/cuda-learn/include/python3.12 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 -c /home/ksharma/dev/git/cuda-learn/build/main.cpp -o main.o 
[2/3] /home/ksharma/anaconda3/envs/cuda-learn/bin/nvcc --generate-dependencies-with-compile --dependency-output cuda.cuda.o.d -DTORCH_EXTENSION_NAME=saxpy_extension -DT

Loading extension module saxpy_extension...


CUDA Runtime version: 11070
CUDA Driver version: 12020


In [6]:
x = torch.randn((2, 3), device="cuda")
y = torch.randn((2, 3), device="cuda")
print(x)
print(y)

saxpy_extension.saxpy_wrapper(x, y, 2.0)

tensor([[ 0.1994,  1.0400,  1.0288],
        [ 0.4695, -0.1891,  0.3379]], device='cuda:0')
tensor([[-1.5099,  0.4243, -1.9025],
        [ 0.7954, -0.9438,  0.2487]], device='cuda:0')
Calculated saxpy


tensor([[-1.1112,  2.5043,  0.1552],
        [ 1.7344, -1.3219,  0.9246]], device='cuda:0')

# Pytorch Profiler

In [7]:
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    saxpy_extension.saxpy_wrapper(x, y, 2.0)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
         cudaLaunchKernel        99.31%       4.149ms        99.31%       4.149ms       4.149ms             1  
    cudaDeviceSynchronize         0.69%      29.000us         0.69%      29.000us      14.500us             2  
-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
Self CPU time total: 4.178ms

Calculated saxpy


STAGE:2024-03-28 07:16:35 431270:431270 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-03-28 07:16:35 431270:431270 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-03-28 07:16:35 431270:431270 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [8]:
import torch
from torch.profiler import profile, record_function, ProfilerActivity
import time


# ## Default way to use profiler
# with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
#     for _ in range(10):
#         a = torch.square(torch.randn(10000, 10000).cuda())

# prof.export_chrome_trace("trace.json")


## With warmup and skip
# https://pytorch.org/docs/stable/profiler.html

# Non-default profiler schedule allows user to turn profiler on and off
# on different iterations of the training loop;
# trace_handler is called every time a new trace becomes available
def trace_handler(prof):
    print(prof.key_averages().table(
        sort_by="self_cuda_time_total", row_limit=-1))
    prof.export_chrome_trace(f"/tmp/{int(time.time())}_test_trace_" + str(prof.step_num) + ".json")

with torch.profiler.profile(
    activities=[
        torch.profiler.ProfilerActivity.CPU,
        torch.profiler.ProfilerActivity.CUDA,
    ],

    # In this example with wait=1, warmup=1, active=2, repeat=1,
    # profiler will skip the first step/iteration,
    # start warming up on the second, record
    # the third and the forth iterations,
    # after which the trace will become available
    # and on_trace_ready (when set) is called;
    # the cycle repeats starting with the next step

    schedule=torch.profiler.schedule(
        wait=1,
        warmup=1,
        active=2,
        repeat=1),
    on_trace_ready=trace_handler
    # on_trace_ready=torch.profiler.tensorboard_trace_handler('./log')
    # used when outputting for tensorboard
    ) as p:
        for iter in range(10):
            saxpy_extension.saxpy_wrapper(torch.randn(10000, 10000).cuda(), torch.randn(10000, 10000).cuda(), 2.0)
            # send a signal to the profiler that the next iteration has started
            p.step()

Calculated saxpy


STAGE:2024-03-28 07:16:50 431270:431270 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


Calculated saxpy
Calculated saxpy
Calculated saxpy
-------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                 Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
    saxpy(int, float, float*, float*)         0.00%       0.000us         0.00%       0.000us       0.000us     230.632ms        57.62%     230.632ms     115.316ms             2  
                          aten::copy_         0.00%      75.000us         6.07%     170.139ms      42.535ms     169.616ms        42.38%     169.616ms      42.404ms             4  
     Memcpy HtoD (Pageable -> Device)         0.

STAGE:2024-03-28 07:16:53 431270:431270 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-03-28 07:16:53 431270:431270 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


Calculated saxpy
Calculated saxpy
Calculated saxpy
Calculated saxpy
Calculated saxpy
Calculated saxpy
